In [ ]:
pip install scrapy selenium
!wget https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chrome-linux64.zip
!wget https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chromedriver-linux64.zip
!unzip chrome-linux64.zip
!unzip chromedriver-linux64.zip
!rm *.zip

In [ ]:
import re
import os
import sys
import json
import time
import scrapy
import warnings
from datetime import datetime
from urllib.parse import urljoin
warnings.filterwarnings("ignore")

from twisted.internet import reactor
from scrapy.crawler import CrawlerRunner
from multiprocessing import Process, Queue


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.binary_location = './chrome-linux64/chrome'
options.add_argument('--no-sandbox')
options.add_argument('--headless')
service = Service(executable_path='./chromedriver-linux64/chromedriver')


# this function gets the data from the spider's callback function and
# saves it to the out_filename (in the specified format out_filetype)
# (no changes necessary to this function)
def run_spider(spider, out_filename, out_filetype):
    if os.path.exists(out_filename):
        os.remove(out_filename)

    def f(q):
        try:
            runner = CrawlerRunner(settings={
                "FEEDS": {
                    out_filename: {
                        "format": out_filetype},
                },
            })
            deferred = runner.crawl(spider)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()
    if result is not None:
        raise result


In [ ]:
class WSJSpider(scrapy.Spider):

    name = "Charlotte" # name your spider

    # list the url(s) where to start reading
    def start_requests(self):
        url = "https://seekingalpha.com/earnings/earnings-call-transcripts"
        yield scrapy.Request(url=url, callback=self.parse)

    # the callback function (called when the webpage is fetched)
    def parse(self, response):
        print(response.css("html").get())   # prints the contents of the webpage on the console

        # write the code for extracting all quotes, author and tags from the webpage (hint use a loop!)
        # see the documentation here: https://docs.scrapy.org/en/latest/topics/selectors.html
        # for i, quote in enumerate(response.css("div.quote")):
        #   text = quote.css("span.text::text").get()[1:-1]
        #   author = quote.css("small.author::text").get()
        #   tags = quote.css("div.tags a.tag::text").getall()
        #   yield {
        #       "text": text,
        #       "author": author,
        #       "tags": tags,
        #   }

        # # once all the quotes are extracted from this page, extract the next page
        # next_page = response.css("li.next a::attr(href)").get()    # extract the link to the next webpage
        # if next_page is not None:
        #     yield response.follow(next_page, callback=self.parse) # to follow the link, ans use the same function


run_spider(MyAllQuotesJSSpider, "all_quotes_js.json", "json")
